In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import seaborn as sns
import numpy as np
import h5py

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import griddata
from collections import OrderedDict

sns.set_style("white")

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
Re=20
layers = [2, 20, 20, 20, 20, 20, 3]
pinn = MLP(layers)
pinn.load_state_dict(torch.load('data/weights/weights_'+str(Re)+'.pt'))

In [ ]:
class HyperNetwork(nn.Module):
    def __init__(self):
        super(Hyper_Model, self).__init__()
        
        self.layers = nn.Sequential(nn.Linear(1,512),
                      nn.ReLU(),             
                      nn.Linear(512,256),
                      nn.ReLU(),
                      nn.Linear(256,256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128,1803))    
        
    def forward(self, Re):        
        weights = self.layers(Re)
        return weights